# Notebook 1 : Setup Environment
CropHealth Detection - Préparation de l'environnement Google Colab

Cellule 1 : Vérifier GPU

In [1]:
import torch
import sys

print("="*70)
print("🔧 CONFIGURATION GPU")
print("="*70)

# GPU Check
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    cuda_version = torch.version.cuda

    print(f"✅ GPU Disponible")
    print(f"   • Modèle: {gpu_name}")
    print(f"   • Mémoire: {gpu_memory:.1f} GB")
    print(f"   • CUDA Version: {cuda_version}")
    print(f"   • PyTorch Version: {torch.__version__}")
else:
    print("❌ Pas de GPU détecté !")
    print("💡 Activer GPU: Runtime > Change runtime type > GPU (T4)")
    sys.exit(1)

print("="*70)

🔧 CONFIGURATION GPU
❌ Pas de GPU détecté !
💡 Activer GPU: Runtime > Change runtime type > GPU (T4)


SystemExit: 1

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Cellule 2 : Cloner le repository

In [ ]:
import os
from pathlib import Path

print("\n" + "="*70)
print("📦 CLONAGE DU PROJET")
print("="*70 + "\n")

# Supprimer ancien clone si existe
if Path('CropHealth_Detection_PFE').exists():
    print("⚠️  Projet déjà cloné, suppression...")
    !rm -rf CropHealth_Detection_PFE

# Cloner
!git clone https://github.com/borisbob91/CropHealth_Detection_PFE.git

# Aller dans le dossier
%cd CropHealth_Detection_PFE

# Vérifier structure
print("\n✅ Projet cloné !")
print("\n📁 Structure du projet:")
!ls -la

## 📌 Cellule 3 : Installer les dépendances

In [ ]:
print("\n" + "="*70)
print("📦 INSTALLATION DES DÉPENDANCES")
print("="*70 + "\n")

# Installation silencieuse
print("⏳ Installation en cours (2-3 minutes)...")

# Core dependencies
!pip install -q torch torchvision torchmetrics

# Detection frameworks
!pip install -q ultralytics effdet timm

# Data & Augmentation
!pip install -q albumentations opencv-python

# Metrics & Evaluation
!pip install -q pycocotools scikit-learn

# Visualization
!pip install -q matplotlib seaborn tensorboard

# Model analysis
!pip install -q thop torchinfo

print("\n✅ Dépendances installées !\n")

# Vérifier versions
import torch
import torchvision
import ultralytics
import albumentations

print("📋 Versions installées:")
print(f"   • PyTorch: {torch.__version__}")
print(f"   • TorchVision: {torchvision.__version__}")
print(f"   • Ultralytics: {ultralytics.__version__}")
print(f"   • Albumentations: {albumentations.__version__}")

# Cellule 4 : Monter Google Drive

In [ ]:
from google.colab import drive

print("\n" + "="*70)
print("💾 MONTAGE GOOGLE DRIVE")
print("="*70 + "\n")

# Monter Drive
drive.mount('/content/drive')

print("\n✅ Google Drive monté !")
print(f"📁 Accessible via: /content/drive/MyDrive/")

## 📌 Cellule 5 : Uploader et préparer le dataset

In [ ]:
from google.colab import files
import zipfile
import shutil

print("\n" + "="*70)
print("📤 PRÉPARATION DU DATASET")
print("="*70 + "\n")

# Option A : Depuis Google Drive
use_drive = True  # Mettre False pour uploader manuellement

if use_drive:
    print("📁 Utilisation du dataset depuis Google Drive...")

    # Adapter ce chemin selon votre structure Drive
    drive_dataset_path = "/content/drive/MyDrive/CropHealth_Dataset"

    if Path(drive_dataset_path).exists():
        # Créer lien symbolique
        !ln -sf {drive_dataset_path} data/pascal_voc
        print(f"✅ Dataset lié depuis: {drive_dataset_path}")
    else:
        print(f"❌ Dataset non trouvé dans Drive: {drive_dataset_path}")
        print("💡 Mettre use_drive=False pour upload manuel")

else:
    # Option B : Upload manuel
    print("📤 Upload votre dataset ZIP (Pascal VOC format)...")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n📦 Extraction de {filename}...")

        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('data/pascal_voc')

        print(f"✅ Extraction terminée !")

# Vérifier structure
print("\n📋 Vérification de la structure...")
!tree data/pascal_voc -L 2 -d

📌 Cellule 6 : Convertir Pascal VOC → YOLO

In [ ]:
print("\n" + "="*70)
print("🔄 CONVERSION PASCAL VOC → YOLO")
print("="*70 + "\n")

# Convertir VOC → YOLO
!python utils/pascalvoc_to_yolo.py \
    --voc-root data/pascal_voc \
    --output data/yolo_crop \
    --format yolo

print("\n✅ Conversion YOLO terminée !")

# 📌 Cellule 7 : Convertir Pascal VOC → COCO

In [ ]:
print("\n" + "="*70)
print("🔄 CONVERSION PASCAL VOC → COCO")
print("="*70 + "\n")

# Convertir VOC → COCO (pour EfficientDet)
!python utils/pascalvoc_to_yolo.py \
    --voc-root data/pascal_voc \
    --output data/coco_crop \
    --format coco

print("\n✅ Conversion COCO terminée !")

📌 Cellule 8 : Vérifier les datasets

In [ ]:
import pandas as pd
from pathlib import Path

print("\n" + "="*70)
print("📊 VÉRIFICATION DES DATASETS")
print("="*70 + "\n")

datasets_info = []

# Pascal VOC
for split in ['train', 'val', 'test']:
    voc_imgs = Path(f'data/pascal_voc/{split}/images')
    voc_anns = Path(f'data/pascal_voc/{split}/Annotations')

    if voc_imgs.exists():
        img_count = len(list(voc_imgs.glob('*.jpg')))
        xml_count = len(list(voc_anns.glob('*.xml'))) if voc_anns.exists() else 0
        datasets_info.append(['Pascal VOC', split, img_count, xml_count])

# YOLO
for split in ['train', 'val', 'test']:
    yolo_imgs = Path(f'data/yolo_crop/{split}/images')
    yolo_lbls = Path(f'data/yolo_crop/{split}/labels')

    if yolo_imgs.exists():
        img_count = len(list(yolo_imgs.glob('*.jpg')))
        txt_count = len(list(yolo_lbls.glob('*.txt'))) if yolo_lbls.exists() else 0
        datasets_info.append(['YOLO', split, img_count, txt_count])

# COCO
for split in ['train', 'val', 'test']:
    coco_imgs = Path(f'data/coco_crop/{split}/images')
    coco_json = Path(f'data/coco_crop/{split}/annotations.json')

    if coco_imgs.exists():
        img_count = len(list(coco_imgs.glob('*.jpg')))
        json_exists = '✅' if coco_json.exists() else '❌'
        datasets_info.append(['COCO', split, img_count, json_exists])

# Afficher tableau
df = pd.DataFrame(datasets_info, columns=['Format', 'Split', 'Images', 'Annotations'])
print(df.to_string(index=False))

print("\n" + "="*70)

viz data

In [ ]:
print("\n" + "="*70)
print("🔄 CONVERSION PASCAL VOC → COCO")
print("="*70 + "\n")

# Convertir VOC → COCO (pour EfficientDet)
!python utils/pascalvoc_to_yolo.py \
    --voc-root data/pascal_voc \
    --output data/coco_crop \
    --format coco

!python utils/yol_viz.py -d data/yolo_crop -n 6

!python utils/pascal_viz.py -d data/pascal_voc -n 6

print("\n✅ Conversion COCO terminée !")

# 📌 Cellule 9 : Créer dossiers de sortie

In [ ]:
print("\n" + "="*70)
print("📁 CRÉATION DES DOSSIERS DE SORTIE")
print("="*70 + "\n")

# Créer structure
output_dirs = [
    'runs',
    'predictions',
    'evaluation_results',
    'exports',
]

for dir_name in output_dirs:
    Path(dir_name).mkdir(exist_ok=True)
    print(f"✅ {dir_name}/")

print("\n✅ Dossiers créés !")

visualiser les données:

## Cellule 10 : Résumé de l'environnement

In [ ]:
import json

print("\n" + "="*70)
print("📋 RÉSUMÉ DE L'ENVIRONNEMENT")
print("="*70 + "\n")

summary = {
    "GPU": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None",
    "CUDA": torch.version.cuda if torch.cuda.is_available() else "N/A",
    "PyTorch": torch.__version__,
    "Datasets": {},
    "Projet": "CropHealth_Detection_PFE",
    "Status": "✅ Prêt pour le training"
}

# Compter images
for format_name in ['pascal_voc', 'yolo_crop', 'coco_crop']:
    train_imgs = Path(f'data/{format_name}/train')
    if train_imgs.exists():
        if format_name == 'pascal_voc':
            count = len(list((train_imgs / 'JPEGImages').glob('*.jpg')))
        else:
            count = len(list((train_imgs / 'images').glob('*.jpg')))

        summary["Datasets"][format_name] = f"{count} images"

# Afficher
print(json.dumps(summary, indent=2, ensure_ascii=False))

print("\n" + "="*70)
print("✅ ENVIRONNEMENT PRÊT !")
print("="*70)
print("\n➡️  Passez au Notebook 2 : Training")